### Distancia de edición mínima

Gran parte del procesamiento del lenguaje natural se centra en medir cuán similares son dos cadenas. Por ejemplo, en la corrección ortográfica, si un usuario escribe una cadena incorrecta, digamos **graffe** y queremos inferir la palabra que realmente quiso escribir, podemos buscar palabras candidatas similares.
Entre estas, la palabra **giraffe**, que difiere en solo una letra de **graffe**, parece intuitivamente más cercana que, por ejemplo, **grail** o **graf**, que difieren en más letras.

Otro ejemplo proviene de la **correferencia**, la tarea de determinar si dos cadenas se refieren a la misma entidad. Consideremos los siguientes ejemplos:

```plaintext
Stanford President Marc Tessier-Lavigne
Stanford University President Marc Tessier-Lavigne
```

El hecho de que estas dos cadenas sean muy similares (diferentes solo en una palabra) proporciona una fuerte evidencia de que podrían ser correferentes.

La **distancia de edición** nos ofrece una forma cuantitativa de evaluar la similitud entre cadenas. Más formalmente, la **distancia de edición mínima** entre dos cadenas se define como el número mínimo de operaciones de edición necesarias para transformar una cadena en otra. Estas operaciones incluyen inserción, eliminación y sustitución de caracteres.

Por ejemplo, la distancia entre "intention" y "execution" es 5, ya que podemos transformar la primera en la segunda mediante las siguientes operaciones:

- Eliminar una `i`
- Sustituir `e` por `n`
- Sustituir `x` por `t`
- Insertar `c`
- Sustituir `u` por `n`

También podemos asignar un costo o peso a cada una de estas operaciones. La **distancia de Levenshtein** es una de las métricas más utilizadas y asigna un costo uniforme de 1 a cada operación (inserción, eliminación o sustitución).

Asumimos que la sustitución de una letra por sí misma, por ejemplo, `t` por `t`, tiene un costo de cero. Según esta métrica, la distancia de Levenshtein entre "intention" y "execution" es 5.

Levenshtein también propuso una variante en la que solo se permiten inserciones y eliminaciones, ambas con un costo de 1, sin permitir sustituciones. Esto es equivalente a permitir sustituciones con un costo de 2, ya que cualquier sustitución puede representarse mediante una inserción seguida de una eliminación. Con esta versión, la distancia entre "intention" y "execution" es 8.


### El algoritmo de la distancia de edición mínima

¿Cómo encontramos la **distancia de edición mínima**? Primero, definamos formalmente este concepto.

Dadas dos cadenas, la cadena fuente **X** de longitud **n** y la cadena objetivo **Y** de longitud **m**, definimos **D[i, j]** como la **distancia de edición** entre los primeros **i** caracteres de **X** (**X[1..i]**) y los primeros **j** caracteres de **Y** (**Y[1..j]**). Por lo tanto, la **distancia de edición** entre **X** y **Y** es **D[n, m]**.

Para calcular **D[n, m]**, utilizamos **programación dinámica**, resolviendo el problema de abajo hacia arriba al combinar soluciones a subproblemas más pequeños. 

#### Caso base:
1. Si la cadena fuente tiene longitud **i** y la cadena objetivo está vacía, se necesitan **i** eliminaciones para transformar **X** en **Y**.
2. Si la cadena objetivo tiene longitud **j** y la cadena fuente está vacía, se necesitan **j** inserciones para convertir **X** en **Y**.

#### Caso general:
Después de calcular **D[i, j]** para valores pequeños de **i** y **j**, utilizamos estos valores para calcular valores mayores de **D[i, j]**. El valor de **D[i, j]** se obtiene tomando el mínimo entre tres posibles caminos en la matriz de distancias:

```plaintext
D[i, j] = min {
    D[i - 1, j] + del-cost(source[i]),    # Eliminación
    D[i, j - 1] + ins-cost(target[j]),    # Inserción
    D[i - 1, j - 1] + sub-cost(source[i], target[j])  # Sustitución
}
```

Previamente, mencionamos dos versiones de la **distancia de Levenshtein**:
- **Versión 1:** Las sustituciones tienen un costo de 1.
- **Versión 2:** Las sustituciones tienen un costo de 2 (equivalente a realizar una inserción seguida de una eliminación).

Aquí utilizaremos la segunda versión, en la que:
- **Las inserciones y eliminaciones tienen un costo de 1** (**ins-cost(·) = del-cost(·) = 1**).
- **Las sustituciones tienen un costo de 2**, excepto cuando los caracteres son idénticos, en cuyo caso el costo es 0.

Bajo esta versión de **Levenshtein**, la ecuación de recurrencia queda:

```plaintext
D[i, j] = min {
    D[i - 1, j] + 1,  # Eliminación
    D[i, j − 1] + 1,  # Inserción
    D[i - 1, j - 1] + {
        2, si source[i] ≠ target[j]  # Sustitución
        0, si source[i] = target[j]  # Sin cambio
    }
}
```

El siguiente algoritmo resume la implementación de la **distancia de edición mínima** y muestra los resultados para calcular la distancia entre *"intention"* y *"execution"* usando la versión de **Levenshtein** mencionada:

```plaintext
function MIN-EDIT-DISTANCE (source, target) returns min-distance
    n <- LENGTH (source)
    m <- LENGTH (target)
    Create a distance matrix D[n+1, m+1]

    # Inicialización: primera fila y primera columna representan la distancia a la cadena vacía
    D[0,0] = 0
    for each row i from 1 to n do
        D[i,0] <- D[i-1,0] + del-cost(source[i])
    for each column j from 1 to m do
        D[0,j] <- D[0, j-1] + ins-cost(target[j])

    # Relación de recurrencia:
    for each row i from 1 to n do
        for each column j from 1 to m do
            D[i, j] ← MIN (
                D[i−1, j] + del-cost(source[i]),         # Eliminación
                D[i−1, j−1] + sub-cost(source[i], target[j]),  # Sustitución
                D[i, j−1] + ins-cost(target[j])          # Inserción
            )

    # Retornar la distancia mínima calculada
    return D[n,m]
```

Este algoritmo pertenece a la categoría de algoritmos de **programación dinámica**. Los costos pueden ser fijos (por ejemplo, **∀x, ins-cost(x) = 1**) o específicos para cada letra, modelando el hecho de que algunas letras tienen más probabilidades de ser insertadas o eliminadas que otras. En esta implementación, asumimos que no hay costo por sustituir una letra por sí misma (**sub-cost(x, x) = 0**).


In [ ]:
def levenshtein_distance(s1, s2):
    """
    Calcula la distancia de Levenshtein estándar entre dos secuencias.
    """
    len_s1 = len(s1) + 1
    len_s2 = len(s2) + 1

    # Crear una matriz para almacenar los resultados parciales
    dp = [[0] * len_s2 for _ in range(len_s1)]

    # Inicializar la primera fila y columna
    for i in range(len_s1):
        dp[i][0] = i
    for j in range(len_s2):
        dp[0][j] = j

    # Llenar la matriz
    for i in range(1, len_s1):
        for j in range(1, len_s2):
            if s1[i - 1] == s2[j - 1]:
                cost = 0
            else:
                cost = 1

            dp[i][j] = min(dp[i - 1][j] + 1,       # Eliminación
                           dp[i][j - 1] + 1,       # Inserción
                           dp[i - 1][j - 1] + cost) # Sustitución

    return dp[-1][-1]

def levenshtein_distance_no_substitution(s1, s2):
    """
    Calcula la distancia de Levenshtein con un costo de sustitución de 2.
    """
    len_s1 = len(s1) + 1
    len_s2 = len(s2) + 1

    # Crear una matriz para almacenar los resultados parciales
    dp = [[0] * len_s2 for _ in range(len_s1)]

    # Inicializar la primera fila y columna
    for i in range(len_s1):
        dp[i][0] = i
    for j in range(len_s2):
        dp[0][j] = j

    # Llenar la matriz
    for i in range(1, len_s1):
        for j in range(1, len_s2):
            if s1[i - 1] == s2[j - 1]:
                cost = 0
            else:
                cost = 2  # Costo de sustitución como inserción + eliminación

            dp[i][j] = min(dp[i - 1][j] + 1,       # Eliminación
                           dp[i][j - 1] + 1,       # Inserción
                           dp[i - 1][j - 1] + cost) # Sustitución

    return dp[-1][-1]

# Ejemplos de uso
s1 = "intention"
s2 = "execution"

distancia_levenshtein = levenshtein_distance(s1, s2)
distancia_no_substitucion = levenshtein_distance_no_substitution(s1, s2)

print(f"Distancia de Levenshtein estándar entre '{s1}' y '{s2}': {distancia_levenshtein}")
print(f"Distancia de Levenshtein (sin sustituciones permitidas) entre '{s1}' y '{s2}': {distancia_no_substitucion}")


In [ ]:
def min_edit_distance(source, target):
    """
    Calcula la distancia de edición mínima entre dos cadenas usando programación dinámica.
    También devuelve los pasos detallados para transformar la cadena fuente en la cadena objetivo.
    
    Parameters:
        source (str): La cadena fuente (X) que se quiere transformar.
        target (str): La cadena objetivo (Y) en la que se quiere transformar la fuente.
    
    Returns:
        int: La distancia mínima de edición entre la cadena fuente y la cadena objetivo.
        list: Los pasos detallados de las operaciones realizadas.
    """
    # n es la longitud de la cadena fuente
    n = len(source)
    # m es la longitud de la cadena objetivo
    m = len(target)
    
    # Crear una matriz de (n+1) x (m+1) para almacenar las distancias
    D = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    # Crear una matriz para almacenar las operaciones
    operaciones = [[None for _ in range(m + 1)] for _ in range(n + 1)]
    
    # Inicialización: la primera fila y la primera columna representan la distancia
    # desde la cadena vacía a la cadena parcial.
    for i in range(1, n + 1):
        D[i][0] = i  # Cada paso cuesta 1, que representa una eliminación
        operaciones[i][0] = "delete"
    for j in range(1, m + 1):
        D[0][j] = j  # Cada paso cuesta 1, que representa una inserción
        operaciones[0][j] = "insert"
    
    # Relación de recurrencia: llenar la matriz utilizando las distancias mínimas
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            # Costo de eliminación
            del_cost = D[i - 1][j] + 1
            # Costo de inserción
            ins_cost = D[i][j - 1] + 1
            # Costo de sustitución (0 si son iguales, 2 si son diferentes)
            sub_cost = D[i - 1][j - 1] + (2 if source[i - 1] != target[j - 1] else 0)
            
            # Determinar el camino mínimo y guardar la operación realizada
            if sub_cost <= del_cost and sub_cost <= ins_cost:
                D[i][j] = sub_cost
                if source[i - 1] == target[j - 1]:
                    operaciones[i][j] = "match"
                else:
                    operaciones[i][j] = "substitute"
            elif del_cost < ins_cost:
                D[i][j] = del_cost
                operaciones[i][j] = "delete"
            else:
                D[i][j] = ins_cost
                operaciones[i][j] = "insert"
    
    # Realizar el backtrace para encontrar los pasos
    steps = []
    i, j = n, m
    while i > 0 or j > 0:
        current_op = operaciones[i][j]
        if current_op == "match" or current_op == "substitute":
            steps.append(f"{current_op} '{source[i-1]}' -> '{target[j-1]}'")
            i -= 1
            j -= 1
        elif current_op == "delete":
            steps.append(f"{current_op} '{source[i-1]}'")
            i -= 1
        elif current_op == "insert":
            steps.append(f"{current_op} '{target[j-1]}'")
            j -= 1
    
    steps.reverse()  # Para obtener los pasos en el orden correcto
    return D[n][m], steps

# Ejemplo de uso
source = "intention"
target = "execution"
distance, steps = min_edit_distance(source, target)

print(f"La mínima distancia de edición entre '{source}' y '{target}' es: {distance}")
print("Los pasos para transformar la cadena son:")
for step in steps:
    print(step)


### Algunas definiciones

#### **Backtrace**  
El **backtrace** es el proceso de rastrear hacia atrás en una estructura de datos, generalmente una matriz de programación dinámica, para reconstruir la solución óptima de un problema. En el contexto de la **distancia de edición mínima**, después de llenar la matriz con los costos de las operaciones de edición, el **backtrace** comienza en la celda final (que contiene el costo total mínimo para transformar una cadena en otra) y sigue los punteros hacia atrás a través de la matriz. Esto permite reconstruir la secuencia exacta de operaciones (inserciones, eliminaciones, sustituciones) que llevaron a esa solución óptima.

#### **Backpointer**  
Un **backpointer** es un puntero o referencia almacenado en una celda de la matriz de programación dinámica que indica la celda anterior desde la cual se obtuvo el valor actual. En otras palabras, un **backpointer** señala qué operación (inserción, eliminación o sustitución) resultó ser la más óptima para alcanzar ese valor. Durante el proceso de **backtrace**, se siguen estos **backpointers** para reconstruir la secuencia de pasos de la solución óptima. Es posible que una celda tenga más de un **backpointer** si varias operaciones resultaron en el mismo costo mínimo.

Estas definiciones proporcionan un contexto más claro sobre cómo se utilizan estos conceptos en el algoritmo de **distancia de edición mínima** y en otros algoritmos basados en **programación dinámica**.


#### **Ejemplo**

Supongamos que tenemos dos cadenas:

- **Cadena 1:** `"kitten"`
- **Cadena 2:** `"sitting"`

El objetivo del algoritmo de **distancia de edición mínima** es calcular el número mínimo de operaciones necesarias (**inserciones, eliminaciones y sustituciones**) para transformar la primera cadena en la segunda.

##### **Paso 1: Inicialización de la matriz**

Primero, creamos una matriz donde:

- Las **filas** representan los caracteres de la primera cadena, incluyendo un espacio vacío al inicio.
- Las **columnas** representan los caracteres de la segunda cadena, también comenzando con un espacio vacío.

|   |   | s | i | t | t | i | n | g |
|---|---|---|---|---|---|---|---|---|
|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| k | 1 |   |   |   |   |   |   |   |
| i | 2 |   |   |   |   |   |   |   |
| t | 3 |   |   |   |   |   |   |   |
| t | 4 |   |   |   |   |   |   |   |
| e | 5 |   |   |   |   |   |   |   |
| n | 6 |   |   |   |   |   |   |   |

Cada celda **D[i, j]** representa el costo mínimo de transformar la subcadena `"kitten"[0:i]` en `"sitting"[0:j]`.  
Inicialmente, la **primera fila y la primera columna** se rellenan con valores incrementales, porque convertir una cadena vacía en otra requiere tantas operaciones como el número de caracteres.

##### **Paso 2: Llenado de la matriz**

Llenamos la matriz siguiendo estas reglas:

- Si los caracteres en la posición actual de ambas cadenas son **iguales**, el costo es el mismo que la celda diagonalmente superior izquierda (**sin operación de edición**).
- Si los caracteres son **diferentes**, el costo es **1 + min()** de:
  - La celda superior (**eliminación**).
  - La celda izquierda (**inserción**).
  - La celda diagonalmente superior izquierda (**sustitución**).

Después de aplicar estas reglas, la matriz resultante es:

|   |   | s | i | t | t | i | n | g |
|---|---|---|---|---|---|---|---|---|
|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| k | 1 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| i | 2 | 2 | 1 | 2 | 3 | 4 | 5 | 6 |
| t | 3 | 3 | 2 | 1 | 2 | 3 | 4 | 5 |
| t | 4 | 4 | 3 | 2 | 1 | 2 | 3 | 4 |
| e | 5 | 5 | 4 | 3 | 2 | 2 | 3 | 4 |
| n | 6 | 6 | 5 | 4 | 3 | 3 | 2 | 3 |

El **valor en la celda inferior derecha** (6,7) es **3**, lo que indica que la **distancia de edición mínima** entre `"kitten"` y `"sitting"` es 3.

##### **Paso 3: Uso de backpointers**

Mientras llenamos la matriz, registramos **backpointers**, que indican de qué celda proviene el valor mínimo.  
Estos punteros permiten reconstruir la secuencia óptima de ediciones.

Para cada celda, la operación asociada es:

- **→ Inserción** (valor proviene de la celda izquierda)
- **↓ Eliminación** (valor proviene de la celda superior)
- **↘ Sustitución** (o sin cambio si los caracteres son iguales)

Ejemplo:  
- La celda **D(5,5)** contiene `2` y proviene de **D(4,4)** porque `"t"` en `"kitten"` coincide con `"t"` en `"sitting"`, sin necesidad de sustitución.

##### **Paso 4: Backtrace**

Ahora que la matriz está completa, realizamos el **backtrace** para reconstruir la secuencia óptima de operaciones:

1. **Partimos de la celda (6,7)** con un valor de `3` (**sustitución "n" por "g"**).
2. **Nos movemos a (5,6)** con un valor de `2` (**los caracteres "n" son iguales**).
3. **Nos movemos a (4,5)** con un valor de `2` (**sustitución "e" por "i"**).
4. **Nos movemos a (3,4)** con un valor de `1` (**los caracteres "t" son iguales**).
5. **Nos movemos a (2,3)** con un valor de `1` (**los caracteres "t" son iguales**).
6. **Nos movemos a (1,2)** con un valor de `1` (**sustitución "i" por "s"**).
7. **Finalmente, nos movemos a (0,1)** con un valor de `0` (**inserción de "s" al inicio**).

##### **Resultado final**

La secuencia óptima de operaciones es:

1. **Sustituir "k" por "s"** → `"kitten"` → `"sitten"`
2. **Sustituir "e" por "i"** → `"sitten"` → `"sittin"`
3. **Sustituir "n" por "g"** → `"sittin"` → `"sitting"`

La **distancia de edición mínima** es **3**.


#### **Alineación**

La alineación de cadenas es una técnica fundamental en el procesamiento de voz y lenguaje. En el **reconocimiento de voz**, la alineación basada en la **distancia de edición mínima** se utiliza para calcular la **tasa de error de palabras** (**WER, Word Error Rate**). También desempeña un papel crucial en la **traducción automática**, donde las oraciones en un **corpus paralelo** (un corpus con textos en dos idiomas) deben ser emparejadas de manera precisa.

Para extender el algoritmo de **distancia de edición** y obtener una alineación, seguimos un proceso de dos pasos:

1. **Almacenar backpointers**  
   Adaptamos el algoritmo de **distancia de edición mínima** para registrar **backpointers** en cada celda de la matriz.  
   - Un **backpointer** señala la celda anterior de la que proviene el valor actual.  
   - Algunas celdas pueden tener múltiples **backpointers**, ya que la distancia mínima pudo haber sido obtenida desde más de una celda previa.

2. **Realizar el backtrace**  
   Una vez que la matriz está llena, realizamos un **backtrace**, comenzando desde la última celda (esquina inferior derecha) y siguiendo los punteros hacia atrás hasta la celda inicial.  
   - Cada camino completo entre la celda final y la inicial representa una alineación de **distancia mínima**.  
   - Para cada celda, marcamos de cuál de las tres celdas vecinas proviene su valor, utilizando hasta tres flechas.  
   - La secuencia de celdas resaltadas en negrita representa una posible alineación óptima entre las dos cadenas.

En nuestro ejemplo, utilizamos la **distancia de Levenshtein**, donde:

- **Inserciones y eliminaciones tienen un costo de 1**.
- **Sustituciones tienen un costo de 2**.

Sin embargo, el algoritmo permite asignar **pesos arbitrarios** a las operaciones. Por ejemplo, en **corrección ortográfica**, las sustituciones entre letras adyacentes en un teclado pueden tener menor costo, ya que son más probables.


#### **Extensión a modelos probabilísticos: el algoritmo de Viterbi**  
El **algoritmo de Viterbi** es una extensión probabilística de la **distancia de edición mínima**. En lugar de calcular la alineación con la menor distancia de edición, **Viterbi** encuentra la **alineación de máxima probabilidad** entre dos secuencias. Este enfoque es especialmente útil en **modelos ocultos de Markov (HMMs)** y en tareas como el **etiquetado de secuencias** en procesamiento del lenguaje natural.


#### Ejemplos

#### **Ejemplo 1: Distancia de edición mínima (Levenshtein)**

Supongamos que queremos alinear las siguientes dos cadenas utilizando la **distancia de edición mínima**:

- **Cadena 1:** `"gato"`
- **Cadena 2:** `"pato"`

El objetivo es calcular el número mínimo de operaciones (**inserciones, eliminaciones y sustituciones**) necesarias para transformar `"gato"` en `"pato"`.

##### **Paso 1: Creación de la matriz de edición**

Construimos una matriz donde:

- Las **filas** representan los caracteres de la primera cadena (`"gato"`).
- Las **columnas** representan los caracteres de la segunda cadena (`"pato"`).
- Inicialmente, llenamos la **primera fila y la primera columna** con valores incrementales, ya que convertir una cadena vacía en otra requiere tantas operaciones como el número de caracteres.

|   |   | p | a | t | o |
|---|---|---|---|---|---|
|   | 0 | 1 | 2 | 3 | 4 |
| g | 1 |   |   |   |   |
| a | 2 |   |   |   |   |
| t | 3 |   |   |   |   |
| o | 4 |   |   |   |   |


##### **Paso 2: Llenado de la matriz**

Para calcular cada celda **D[i, j]**, seguimos estas reglas:

- Si los caracteres en la posición actual son **iguales**, tomamos el valor de la celda diagonal superior izquierda (**sin costo adicional**).
- Si son **diferentes**, tomamos el mínimo de:
  - **La celda superior** (`D[i-1, j]` → eliminación).
  - **La celda izquierda** (`D[i, j-1]` → inserción).
  - **La celda diagonal superior izquierda** (`D[i-1, j-1]` → sustitución).  

A este valor mínimo, sumamos **1**, ya que es el costo de realizar la operación.

Llenamos la matriz siguiendo estas reglas:

|   |   | p | a | t | o |
|---|---|---|---|---|---|
|   | 0 | 1 | 2 | 3 | 4 |
| g | 1 | 1 | 2 | 3 | 4 |
| a | 2 | 2 | 1 | 2 | 3 |
| t | 3 | 3 | 2 | 1 | 2 |
| o | 4 | 4 | 3 | 2 | 1 |

El valor en la celda **(4,4) = 1** nos indica que la **distancia de edición mínima** entre `"gato"` y `"pato"` es **1**.

##### **Paso 3: Uso de backpointers**

Mientras llenamos la matriz, registramos **backpointers**, que indican de dónde proviene el valor mínimo en cada celda. Estos punteros permiten reconstruir la secuencia óptima de ediciones.

Ejemplos de backpointers en la matriz:

- **Celda (2,2)**: Apunta a (1,1) porque `"a"` en `"gato"` y `"a"` en `"pato"` coinciden (**sin costo**).
- **Celda (3,3)**: Apunta a (2,2) porque `"t"` en ambas cadenas coincide (**sin costo**).


##### **Paso 4: Backtrace**

El **backtrace** comienza en la celda inferior derecha **(4,4)** y sigue los **backpointers** hasta la celda inicial **(0,0)**, reconstruyendo la secuencia de operaciones:

1. **(4,4) →** `"o"` coincide, **sin operación**.
2. **(3,3) →** `"t"` coincide, **sin operación**.
3. **(2,2) →** `"a"` coincide, **sin operación**.
4. **(1,1) →** `"g"` y `"p"` no coinciden, **sustitución** de `"g"` por `"p"`.

**Resultado final:**  
- **Transformación de `"gato"` en `"pato"`** mediante **una única sustitución** (`"g" → "p"`).  
- **Distancia de edición mínima = 1**.

##### **Paso 5: Extensión con el algoritmo de Viterbi**

El **algoritmo de Viterbi** es una extensión probabilística de la **distancia de edición mínima**.  
En lugar de calcular solo el número mínimo de operaciones, **Viterbi** asigna probabilidades a cada posible transición (**inserción, eliminación o sustitución**) y encuentra la **secuencia de máxima probabilidad**.

Ejemplo de asignación de probabilidades:

| Operación       | Probabilidad |
|----------------|-------------|
| Coincidencia   | 0.80        |
| Sustitución    | 0.10        |
| Inserción      | 0.05        |
| Eliminación    | 0.05        |

En lugar de usar costos constantes (como `1` para inserciones/eliminaciones y `2` para sustituciones), **Viterbi** busca **maximizar la probabilidad total** de una secuencia de ediciones, lo que lo hace útil en tareas como **reconocimiento de voz** y **alineación de secuencias en NLP**.



#### **Ejemplo 2: Alineación con el algoritmo de Viterbi**

Para alinear las cadenas `"gato"` y `"pato"`, podemos calcular la **probabilidad de la secuencia de transformaciones** en lugar de solo contar las operaciones de edición mínima.

##### **Secuencia de transformaciones y probabilidades:**

1. **Sustitución de `"g"` por `"p"`** → Probabilidad = **0.1**
2. **Coincidencia de `"a"` con `"a"`** → Probabilidad = **0.8**
3. **Coincidencia de `"t"` con `"t"`** → Probabilidad = **0.8**
4. **Coincidencia de `"o"` con `"o"`** → Probabilidad = **0.8**

La **probabilidad total de esta alineación** es:

$$
0.1 \times 0.8 \times 0.8 \times 0.8 = 0.0512
$$

El **algoritmo de Viterbi** busca **maximizar** esta probabilidad, seleccionando la alineación con la mayor probabilidad total, en lugar de simplemente minimizar la cantidad de operaciones.

##### **Resumen**

- **Matriz de edición:** Representa los costos o probabilidades acumuladas de transformar una cadena en otra.  
- **Backpointers:** Indican la celda anterior desde la que se derivó el valor actual (inserción, eliminación o sustitución).  
- **Backtrace:** Siguiendo los backpointers desde la celda final hasta la inicial, podemos reconstruir la secuencia de operaciones óptima.  
- **Distancia de edición mínima:** Mide el número mínimo de operaciones necesarias para transformar una cadena en otra.  
- **Algoritmo de Viterbi:** Una extensión probabilística que encuentra la alineación más probable en lugar de la alineación de costo mínimo.  


In [ ]:
def alineacion_viterbi(source, target, match_prob, mismatch_prob, gap_prob):
    """
    Calcula la "alineación de máxima probabilidad" entre dos cadenas usando una variante del algoritmo de Viterbi.
    
    Parameters:
        source (str): La cadena fuente que se quiere alinear.
        target (str): La cadena objetivo a la que se quiere alinear la fuente.
        match_prob (float): La probabilidad de una coincidencia exacta entre caracteres.
        mismatch_prob (float): La probabilidad de una sustitución entre caracteres diferentes.
        gap_prob (float): La probabilidad de insertar o eliminar un carácter.
    
    Returns:
        float: La probabilidad máxima de alineación entre las dos cadenas.
        list: Los pasos detallados de las operaciones realizadas en la alineación.
    """
    # n es la longitud de la cadena fuente
    n = len(source)
    # m es la longitud de la cadena objetivo
    m = len(target)
    
    # Crear una matriz de (n+1) x (m+1) para almacenar las probabilidades logarítmicas
    V = [[-float('inf') for _ in range(m + 1)] for _ in range(n + 1)]
    # Crear una matriz para almacenar las operaciones
    operaciones = [[None for _ in range(m + 1)] for _ in range(n + 1)]
    
    # Inicialización: la primera fila y la primera columna representan la alineación
    # desde la cadena vacía a la cadena parcial, penalizando con logaritmo del gap.
    V[0][0] = 0  # Probabilidad logarítmica inicial de 0 (log(1) = 0)
    for i in range(1, n + 1):
        V[i][0] = V[i-1][0] + gap_prob
        operaciones[i][0] = "delete"
    for j in range(1, m + 1):
        V[0][j] = V[0][j-1] + gap_prob
        operaciones[0][j] = "insert"
    
    # Relación de recurrencia: llenar la matriz utilizando las probabilidades logarítmicas
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            # Cálculo de las probabilidades logarítmicas
            match_or_mismatch = V[i-1][j-1] + (match_prob if source[i-1] == target[j-1] else mismatch_prob)
            deletion = V[i-1][j] + gap_prob
            insertion = V[i][j-1] + gap_prob
            
            # Determinar el camino de máxima probabilidad
            if match_or_mismatch >= deletion and match_or_mismatch >= insertion:
                V[i][j] = match_or_mismatch
                operaciones[i][j] = "match" if source[i-1] == target[j-1] else "substitute"
            elif deletion > insertion:
                V[i][j] = deletion
                operaciones[i][j] = "delete"
            else:
                V[i][j] = insertion
                operaciones[i][j] = "insert"
    
    # Realizar el backtrace para encontrar los pasos
    steps = []
    i, j = n, m
    while i > 0 or j > 0:
        current_op = operaciones[i][j]
        if current_op == "match" or current_op == "substitute":
            steps.append(f"{current_op} '{source[i-1]}' -> '{target[j-1]}'")
            i -= 1
            j -= 1
        elif current_op == "delete":
            steps.append(f"{current_op} '{source[i-1]}'")
            i -= 1
        elif current_op == "insert":
            steps.append(f"{current_op} '{target[j-1]}'")
            j -= 1
    
    steps.reverse()  # Para obtener los pasos en el orden correcto
    return V[n][m], steps

# Ejemplo de uso
source = "intention"
target = "execution"
match_prob = 0.0  # log(1) = 0 para coincidencia
mismatch_prob = -2.0  # log(probabilidad de sustitución)
gap_prob = -1.0  # log(probabilidad de gap)

probability, steps = alineacion_viterbi(source, target, match_prob, mismatch_prob, gap_prob)

print(f"La máxima probabilidad logarítmica de alineación entre '{source}' y '{target}' es: {probability}")
print("Los pasos para alinear la cadena son:")
for step in steps:
    print(step)


### Ejercicios

**Ejercicio 1: Distancia de edición entre "leda" y "deal"**

**Pregunta:** Calcula la distancia de edición entre las palabras "leda" y "deal", usando un costo de 1 para inserciones, eliminaciones y sustituciones. Muestra tu trabajo utilizando una cuadrícula de distancia de edición.

**Instrucciones:**
1. Crea una cuadrícula para almacenar las distancias parciales entre los prefijos de "leda" y "deal".
2. Llena la cuadrícula calculando las distancias según los costos dados.
3. Identifica la secuencia de operaciones (inserción, eliminación, sustitución) necesarias para transformar "leda" en "deal".
4. Calcula la distancia mínima de edición.

**Solución esperada:**
- Deberías construir una matriz de 5x5 (ya que ambas palabras tienen 4 letras más una fila y columna para la cadena vacía).
- Debes completar la matriz paso a paso y luego seguir la secuencia óptima para encontrar la distancia mínima.

**Ejercicio 2: Comparación de palabras ("drive", "brief", "divers")**

**Pregunta:** Determina si "drive" está más cerca de "brief" o de "divers" utilizando la distancia de edición. Calcula la distancia de edición entre "drive" y cada una de las palabras "brief" y "divers". Puedes usar cualquier versión de distancia que prefieras.

**Instrucciones:**
1. Implementa o utiliza una función de distancia de edición para calcular la distancia entre "drive" y "brief", y entre "drive" y "divers".
2. Usa cualquier variante de la distancia de edición (por ejemplo, costo de sustitución 1 o 2).
3. Compara los resultados para determinar cuál de las dos palabras está más cerca de "drive".

**Solución esperada:**
- Debes justificar la elección de la variante de distancia de edición y mostrar cómo se calculó cada distancia.
- Deberías poder concluir cuál de las dos palabras es más cercana a "drive" basándose en los resultados.

**Ejercicio 3: Implementación del algoritmo de distancia de edición mínima**

**Pregunta:** Implementa un algoritmo para calcular la distancia de edición mínima entre dos cadenas y utiliza los resultados que has calculado manualmente para verificar tu código.

**Instrucciones:**
1. Implementa una función en el lenguaje de programación de tu elección para calcular la distancia de edición mínima entre dos cadenas.
2. Usa las palabras y los resultados calculados a mano en ejercicios anteriores (como "leda" y "deal") para verificar la exactitud de tu código.
3. Documenta cómo se valida tu implementación utilizando los ejemplos manuales.

**Solución Esperada:**
- Debes escribir un código que siga la metodología de programación dinámica para calcular la distancia de edición mínima.
- Deberías incluir pruebas en tu código utilizando los ejemplos resueltos manualmente.


**Ejercicio 4: Alineación y backtrace**

**Pregunta:** Mejora el algoritmo de distancia de edición mínima para que también devuelva la alineación de las dos cadenas. Deberás almacenar punteros y agregar una etapa para calcular el **backtrace**.

**Instrucciones:**
1. Modifica tu algoritmo de distancia de edición para que además de calcular la distancia, guarde los punteros necesarios para reconstruir la alineación óptima.
2. Implementa una función de **backtrace** que recorra los punteros desde la celda final hasta la inicial para reconstruir la secuencia de operaciones.
3. Muestra cómo tu algoritmo devuelve tanto la distancia mínima de edición como la alineación resultante entre las dos cadenas.

**Solución esperada:**
- El código debería mostrar tanto la distancia mínima de edición como una secuencia detallada de operaciones que transforman una cadena en la otra.
- Debes verificar que la alineación producida coincide con la secuencia de operaciones mínima.


**Ejercicio 5: Distancia de Levenshtein con costos personalizados**

**Pregunta:** Modifica el algoritmo de distancia de Levenshtein para que acepte costos personalizados para cada operación (inserción, eliminación, sustitución). Luego, calcula la distancia entre "kitten" y "sitting" utilizando los siguientes costos:
- Inserción: 1
- Eliminación: 1
- Sustitución: 2

**Instrucciones:**
1. Implementa o modifica la función de distancia de Levenshtein para aceptar costos específicos para cada operación.
2. Calcula la distancia de edición entre "kitten" y "sitting" utilizando los costos dados.
3. Muestra los pasos realizados para obtener la distancia mínima.

**Solución esperada:**
- Debes modificar la función existente o crear una nueva que permita la personalización de costos.
- Debes mostrar el resultado calculado y los pasos tomados.


**Ejercicio 6: Visualización de la matriz de distancia de edición**

**Pregunta:** Implementa una visualización gráfica de la matriz de distancia de edición entre dos cadenas. Usa colores o valores numéricos para mostrar la progresión de las distancias.

**Instrucciones:**
1. Implementa un algoritmo de distancia de edición mínima.
2. Genera la matriz de distancias entre las cadenas "flaw" y "lawn".
3. Crea una visualización (puede ser una matriz de números o una representación gráfica) que muestre cómo se calculan las distancias en cada celda.

**Solución esperada:**
- Debes producir una visualización clara de la matriz, mostrando cómo se propagan las distancias.
- La visualización debería ayudar a entender cómo se llega al valor final.

**Ejercicio 7: Distancia de edición con transformaciones complejas**

**Pregunta:** Considera un caso en el que, además de las operaciones de inserción, eliminación, y sustitución, tienes una operación adicional llamada "transposición", que permite intercambiar dos caracteres adyacentes con un costo de 1. Implementa este nuevo algoritmo y calcula la distancia entre "converse" y "conserve".

**Instrucciones:**
1. Modifica el algoritmo de distancia de edición mínima para incluir la operación de transposición.
2. Calcula la distancia de edición entre "converse" y "conserve".
3. Muestra los pasos realizados y compara los resultados con los que obtendrías sin la operación de transposición.

**Solución esperada:**
- Debes ajustar el algoritmo para manejar transposiciones y demostrar cómo afecta al resultado.
- Se espera que demuestren cómo la inclusión de transposiciones puede reducir la distancia de edición.

**Ejercicio 8: Comparación de algoritmos de distancia de edición**

**Pregunta:** Implementa y compara tres algoritmos de distancia de edición: Levenshtein estándar, Levenshtein sin sustituciones permitidas (cada sustitución cuesta 2), y el algoritmo con transposiciones. Usa estos algoritmos para calcular las distancias entre las siguientes parejas de palabras: "abc" y "acb", "abcdef" y "abcfed", "flaw" y "lawn".

**Instrucciones:**
1. Implementa las tres variantes del algoritmo de distancia de edición.
2. Calcula la distancia de edición para las palabras dadas usando cada variante.
3. Compara y discute los resultados obtenidos.

**Solución esperada:**
- Debes mostrar cómo se calculan las distancias utilizando cada variante y explicar las diferencias en los resultados.
- Debes analizar en qué casos una variante es más útil que las otras.

**Ejercicio 9: Aplicación a la corrección ortográfica**

**Pregunta:** Desarrolla un corrector ortográfico simple basado en la distancia de edición. Dado un diccionario de palabras y una palabra con un error tipográfico, el corrector debe sugerir la palabra correcta más cercana en el diccionario.

**Instrucciones:**
1. Implementa una función que calcule la distancia de edición entre la palabra mal escrita y cada palabra en un diccionario.
2. Devuelve la palabra del diccionario que tiene la menor distancia de edición con la palabra mal escrita.
3. Prueba tu corrector ortográfico con un diccionario pequeño y algunas palabras mal escritas.

**Solución esperada:**
- Debes implementar una solución que encuentre y sugiera la palabra correcta basándose en la distancia de edición mínima.
- Debes mostrar ejemplos de correcciones con palabras reales.


In [ ]:
##Tus respuestas